last edited on February 6, 2019 by Claire Valva
# Simulated seasonal data using ENSO partitions
TO DO: everything/this is still very much a draft, note also that a substantial portion of this code is copied from extendedifft_randomphase.ipynb

## packages/load files

In [1]:
#import packages
import numpy as np
from scipy.fftpack import fft, ifft, fftfreq, fftshift, ifftshift
import matplotlib.pyplot as plt
import scipy.integrate as sciint
import pandas as pd
import matplotlib.cm as cm
import matplotlib.ticker as tck
from math import pi
from sympy import solve, Poly, Eq, Function, exp, re, im
from netCDF4 import Dataset, num2date # This is to read .nc files and time array
from scipy.optimize import fsolve
from IPython.display import display, Markdown, Latex
import matplotlib.colors as colors
from seaborn import cubehelix_palette #for contour plot colors
import seaborn as sns
from cartopy.util import add_cyclic_point
from decimal import Decimal
import pickle
import time
import random

In [2]:
##get names to later match season + year to arrays and seasonal averaging
file_Name = "names_seasons"
file_pickle = open(file_Name,'rb') 
names_matched, indices_matched_time = pickle.load(file_pickle)

In [3]:
#get phases of transforms, that were solved in phase_check notebook
file_Name = "test_phases_nowind"
file_pickle = open(file_Name,'rb') 
file = pickle.load(file_pickle)

In [4]:
#get geopotential height dataset
# Access data store
data_store = pd.HDFStore('processed_data.h5')

# Retrieve data using key
geopot_df = data_store['preprocessed_geopot']
data_store.close()

## convert loaded objects into usable formats, average over seasons

In [5]:
#get zonal spacing array for plotting later
zonal_spacing = fftfreq(240,1.5)
zonal_spacing = 1/zonal_spacing
zonal_spacing= 360 / zonal_spacing

/home/clairev/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
#puts arrays into list formats (not sure exactly why this is necessary)
#however it is, otherwise isn't compatible with modulo operation
tested = [[[list(leaf)[1] for leaf in stem] for stem in trunk] for trunk in file]
phases = [[[np.remainder(leaf, 2*pi) 
            for leaf in stem] 
           for stem in trunk] 
          for trunk in tested]

amps = [[[list(leaf)[0] for leaf in stem] for stem in trunk] for trunk in file]

In [7]:
#sorts phases and amplitudes into seasons, whose index matches list: seasons
seasons = ['winter', 'spring', 'summer', 'fall']

#sort them into each season
d2_seasons = [[phases[i] for i in range(len(phases) - 1) 
               if names_matched[i][1] == part] for part in seasons]
#sort them into each season
season_phases = [[phases[i] for i in range(len(phases)) 
               if names_matched[i][1] == part] for part in seasons]

#sort them into each season
season_amps = [[amps[i] for i in range(len(phases) - 1) 
               if names_matched[i][1] == part] for part in seasons]

## choose and exclude ENSO years
Here I make (rough) lists of el nino/regular/la nina years, pulled from a NOAA webpage (https://www.esrl.noaa.gov/psd/enso/past_events.html) — if this method seems to improve predictions, I will refine criteria used for definition. After that I will then partition the data and compare relative models. Years used are 1979-2016.

In [15]:
## choose and sort ENSO years, see notebook for more details on the details for being chosen
# (although this should be updated soon)

In [8]:
#make lists of el nino/regular/la nina years
nino = [1980,1983,1987,1988,1992,
        1995,1998,2003,2007,2010]
neutral = [1979,1981,1982,1984,1985,1986,1990,
           1991,1993,1994,1996,1997,2001,2002,
           2004,2005,2006,2009,2013,2014,2015,2016]
nina = [1989,1999,2000,2008,2011,2012]

### simulate these separate ENSO years

In [9]:
#sort into those years and seasons and put into lists (surprise surprise)
nino_amps = [[amps[i] for i in range(len(phases) - 1) 
               if names_matched[i][1] == part and names_matched[i][0] in nino] 
               for part in seasons]
neutral_amps = [[amps[i] for i in range(len(phases) - 1) 
               if names_matched[i][1] == part and names_matched[i][0] in neutral] 
               for part in seasons]
nina_amps = [[amps[i] for i in range(len(phases) - 1) 
               if names_matched[i][1] == part and names_matched[i][0] in nina] 
               for part in seasons]

In [11]:
#adjust for winter averaging
#TO DO: come up with better procedure rather 
#current: chopping off edges to make the same length for averaging
norml = 359
longl = 363

def padded(to_pad, index):
    length = len(to_pad)
    if index == 0:
        zeros = longl - length
        to_pad = list(to_pad)
        for i in range(zeros):
            to_pad.append(0)
        return to_pad
    else:
        return to_pad

In [14]:
#pad rows with zeros to account for leap year
nino_amps_adj = [[[padded(row, index = i)  
                     for row in entry] for entry in nino_amps[i]] for i in range(4)]
nina_amps_adj = [[[padded(row, index = i)  
                     for row in entry] for entry in nina_amps[i]] for i in range(4)]
neutral_amps_adj = [[[padded(row, index = i)  
                     for row in entry] for entry in neutral_amps[i]] for i in range(4)]

In [16]:
#get averages for each year type
nino_avgs = [[np.average(season, axis = 0)] for season in nino_amps_adj]
nina_avgs = [[np.average(season, axis = 0)] for season in nina_amps_adj]
neutral_avgs = [[np.average(season, axis = 0)] for season in neutral_amps_adj]

avg_lists = [nino_avgs, nina_avgs, neutral_avgs]
name_list = ["nino", "nina", "neutral"]

In [ ]:
## perform ifft